In [1]:
using NGSIM
using AutomotiveDrivingModels
using AutoViz
using Interact # Make video in notebook
using Reel # Save video as gif
using CSV # For writing to csv
using DataFrames # For writing to csv
using Distributions
using Test
using StatsBase
using Dates
using Plots

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-14308350402476536311\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-1770924072470710399\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio/main\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n        console.warn(\"Loading WebIO Jupyter extension on an ad-hoc basis. Consider enabling the WebIO nbextension for a stabler experience (this should happen automatically when building WebIO).\");\n        require([\"/assetserver/edf674b4a352d3049d1cf8e1b8f9eda882001cd6-dist/main.js\"], function (webIOModule) {\n            webIOModule.load_ipython_extension();\n        });\n        warning_div.innerHTML = \"<strong>Loading WebIO Jupyter extension on an ad-hoc basis. Consider enabling the WebIO nbextension for a stabler experience (this should happen automatically when building WebIO).</strong>\";\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<div\n    id=\"webio-warning-14308350402476536311\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    <!-- TODO: link to installation docs. -->\n</div>\n")

In [2]:
include("admin_functions.jl"); # Brings in the functions defined in admin_functions.jl

# Utils

In [ ]:
"""
# Example
td1 = load_trajdata(1);
scene = Scene(500)
get!(scene,td1,300)
display(scene.entities[2].state.posF.s)
roadway = ROADWAY_101;
particle = Dict(:v_des=>25.0,:σ=>0.5)
hallucinate_a_step(roadway,scene,particle,car_id=scene.entities[2].id)
"""
function hallucinate_a_step(roadway,scene_input,particle;car_id=-1)
    if car_id==-1 @show "Please give valid car_id" end
    
    scene = deepcopy(scene_input)
    #scene = scene_input # This was the failure case
    n_cars = scene.n 

    models = Dict{Int, DriverModel}()
    
    # Create driver models for all the cars in the scene
    for veh in scene
        if veh.id == car_id
            models[veh.id] = IntelligentDriverModel(;particle...)
        else
            # TODO: RESEARCH QUESTION: What drives the other vehicles in the hallucination
            models[veh.id] = IntelligentDriverModel(v_des=10.0)
        end
    end
    
    n_steps = 1
    dt = 0.1
    rec = SceneRecord(n_steps, dt)
    
    simulate!(rec, scene, roadway, models, n_steps)
    
    X = Array{Float64}(undef,n_steps, 1)

    for t in 1:n_steps
        f = rec.frames[n_steps - t + 1]
        
            # Access the vehicle with id as car_id and return its frenet s
        X[t,1] = scene.entities[findfirst(car_id,f)].state.posF.s
    end
    return X[1]
end

In [ ]:
function compute_particle_likelihoods(roadway,f,trupos,p_set_dict;car_id=-1)
    if car_id==-1 @show "Please give valid car_id" end
    timestep = 0.1 #TODO: Remove hardcoding
    p_mat, params, vec_val_vec = to_matrix_form(p_set_dict)
    
    num_params=size(p_mat)[1]
    num_p = size(p_mat)[2]
    lkhd_vec = Array{Float64}(undef,num_p)
    for i in 1:num_p    
        # Create dict version for a single particle
        p_dict = Dict()
        for j in 1:num_params
            p_dict[params[j]]=vec_val_vec[j][i]
        end
        
        std_dev_acc = p_dict[:σ]
        
        # hack to avoid the std_dev_pos become negative and error Normal distb
        if std_dev_acc <= 0 std_dev_acc = 0.1 end
        
        # TODO: This math needs to be verified from random variable calculations
        std_dev_pos = timestep*timestep*std_dev_acc
            
        hpos = hallucinate_a_step(roadway,f,p_dict,car_id=car_id)
        lkhd_vec[i] = pdf(Normal(hpos,std_dev_pos),trupos[1])
    end
    return lkhd_vec,p_mat,params
end

In [ ]:
function update_p_one_step(roadway,f,trupos,p_set_dict;
                            car_id=-1,approach="pf",elite_fraction_percent=20)
    if car_id==-1 @show "Provide valid car_id" end
    
    lkhd_vec,p_mat,params = compute_particle_likelihoods(roadway,f,trupos,p_set_dict,car_id=car_id)
    
    num_params = size(p_mat)[1]
    num_p = size(p_mat)[2]
    
    if approach=="pf"
        p_weight_vec = weights(lkhd_vec./sum(lkhd_vec)) # Convert to weights form to use julia sampling
        idx = sample(1:num_p,p_weight_vec,num_p)
        new_p_mat = p_mat[:,idx] #Careful that idx is (size,1) and not (size,2)
    end
    
    if approach=="cem"
        sortedidx = sortperm(lkhd_vec,rev=true)
        numtop = convert(Int64,ceil(num_p*elite_fraction_percent/100.0))
        best_particles = p_mat[:,sortedidx[1:numtop]] # elite selection
#         @show best_particles
        p_distribution = fit(MvNormal,best_particles) # fit distb using elites
        new_p_mat = rand(p_distribution,num_p) # sample num_p new particles from dist
    end
    
    new_p_set_dict = to_dict_form(params,new_p_mat)
    return new_p_set_dict
end

In [ ]:
"""
    function get_ngsim_scene(framenum::Int64)

Get a specific scene from traj_ngsim

# Example:
scene = get_ngsim_scene(1)
"""
function get_ngsim_scene(framenum::Int64)
    scene = Scene(500)
    get!(scene,traj_ngsim,framenum)
    return scene
end

In [ ]:
"""
    my_overlay
Overlaying hallucinated trajectory on the ground truth
# Fields
- `color::Colorant`
- `scene::Scene`
"""
# Borrowed from visualization_functions.jl
struct my_overlay <: SceneOverlay
    scene::Scene
    color # Needs to be of form colorant"Colorname"
end

function AutoViz.render!(rendermodel::RenderModel,overlay::my_overlay, 
        scene::Scene, roadway::Roadway)
    AutoViz.render!(rendermodel,overlay.scene,car_color = overlay.color)
    return rendermodel
end

In [ ]:
"""
    IDOverlay
Display the ID on top of each entity in a scene.
# Fields
- `color::Colorant`
- `font_size::Int64`
"""
mutable struct IDOverlay <: SceneOverlay
    color::Colorant
    font_size::Int
end

function AutoViz.render!(rendermodel::RenderModel, overlay::IDOverlay, scene::Scene, 
                            env::E) where E
    font_size = overlay.font_size
    for veh in scene
        add_instruction!(rendermodel, render_text, ("$(veh.id)", veh.state.posG.x, 
                        veh.state.posG.y, font_size, overlay.color), incameraframe=true)
    end
    return rendermodel
end

In [ ]:
"""
    function make_video(;range=nothing,filename="output.mp4")

Makes a video of the highd dataset on the highd roadway taking frame range as input

# Example 
```
make_video(range=11:60,filename="highd_11to60.mp4")
```
"""
function make_video(;range=nothing,filename="output.mp4")

    frames = Frames(MIME("image/png"), fps=10)
    scene = Scene(500)
    for i in range
        temp_scene = get!(scene,traj_highd,i)

        overlays = [TextOverlay(text=["$(veh.id)"], incameraframe=true, 
        pos=VecSE2(veh.state.posG.x+0.5,veh.state.posG.y+0.5)) for veh in scene]
        
        scene_visual = render(temp_scene, roadway_highd, 
            #cam=CarFollowCamera{Int}(2,5.0),
            #cam=StaticCamera(VecE2(1966400, 570900), 5.0),
            cam=FitToContentCamera(0.),
            overlays)
        push!(frames,scene_visual)
    end
    write(string("media/"*filename),frames)    
    
    return nothing
end

# Workspace

In [ ]:
td1 = load_trajdata(1)

scene = Scene(500)
temp_scene = get!(scene,td1,1000)
dummy = render(temp_scene,ROADWAY_101);
typeof(dummy)

In [ ]:
# Setting up for video saving using Reel
frames = Frames(MIME("image/png"), fps=10)
for i in 400:500
    temp_scene = get!(scene,td1,i)
    carcolors = Dict{Int,Colorant}()
    for veh in temp_scene
        #@show veh.id
        #carcolors[veh.id] = in(veh.id, egoid) ? colorant"blue" : colorant"green"
        
            # Color cars green when they are in segment 3
        if veh.state.posF.roadind.tag.segment == 3
            carcolors[veh.id] = colorant"green"
        end
        
    end
    overlays = [TextOverlay(text=["$(veh.id)"], incameraframe=true, 
                pos=VecSE2(veh.state.posG.x+0.5,veh.state.posG.y+0.5)) for veh in scene]
    scene_visual = render(temp_scene, ROADWAY_101, 
        #cam=CarFollowCamera{Int}(2,5.0),
        cam=StaticCamera(VecE2(1966400, 570900), 5.0),
        #cam=FitToContentCamera(0.),
        overlays,
        car_colors=carcolors)
    push!(frames,scene_visual)
end
write("ngsim_segment_zoomed.mp4",frames)

In [ ]:
# argument 1 loads i101 7:50 to 8:05.
# load_trajdata function defined in NGSIM.jl/src/trajdata.jl
td1 = load_trajdata(1); 

scene = Scene(500)
egoid = [25,5,8,9,10,12,21] # These vehicle ids will be colored blue if the egoid test is performed

# List of the vehicle ids in the scene
# veh.id = 2 5 8 9 10 12 13 14 18 20 21 22 23 25 26 27 31 32 34 35 39 48

# Drive here in the notebook. Replay the trajectory as recorded in the ngsim data
@manipulate for i in 400:500
    temp_scene = get!(scene,td1,i)
    
    carcolors = Dict{Int,Colorant}()
    for veh in temp_scene
        #@show veh.id
        #carcolors[veh.id] = in(veh.id, egoid) ? colorant"blue" : colorant"green"
        
            # Color cars green when they are in segment 3
        if veh.state.posF.roadind.tag.segment == 3
            carcolors[veh.id] = colorant"green"
        end
        
    end
    overlays = [TextOverlay(text=["$(veh.id)"], incameraframe=true, 
                pos=VecSE2(veh.state.posG.x+0.5,veh.state.posG.y+0.5)) for veh in scene]
    render(temp_scene, ROADWAY_101, 
        #cam=CarFollowCamera{Int}(2,5.0),
        #cam=StaticCamera(VecE2(1966400, 570900), 5.0),
         cam=FitToContentCamera(0.),
        overlays,
        car_colors=carcolors)
end

In [ ]:
for i in 150:160
    scene = Scene(500) # Have to create a new scene container every time, wonder how manipulate got by without
    t = get!(scene,td1,i)
    for veh in t
        if veh.state.posF.roadind.tag.segment == 3
            print("$(veh.id) is in segment 3\n")
        else
            print("$(veh.id) not in segment 3\n")
        #display(t.entities[1].state.posF.roadind.tag.segment)
        end
    end
end

# Hallucination from prerecorded scene

In [ ]:
# Let's take a scene from prerecorded trajdata and try a hallucination from that
scene_input = Scene(500)
get!(scene_input,td1,300)
scene = deepcopy(scene_input)
overlays = [TextOverlay(text=["$(veh.id)"], incameraframe=true, 
                pos=VecSE2(veh.state.posG.x+0.5,veh.state.posG.y+0.5)) for veh in scene];
#render(start_scene, ROADWAY_101, cam=StaticCamera(VecE2(1966400, 570900), 5.0),overlays)

In [ ]:
scene.entities[2].state.posF.s

In [ ]:
n_cars = scene.n
models = Dict{Int,DriverModel}()
for veh in scene
    models[veh.id] = IntelligentDriverModel(v_des=10.0)
end

In [ ]:
n_steps = 1
dt = 0.1
rec = SceneRecord(n_steps,dt)
simulate!(rec,scene,roadway,models,n_steps)

In [ ]:
scene.entities[2].state.posF.s

In [ ]:
rec.frames[1].entities[2].state.posF.s

In [ ]:
# Render the frame after doing the simulation for 1 step
carcolors = Dict{Int,Colorant}()
carcolors[2] = colorant"green"
render(rec.frames[1],roadway,cam=CarFollowCamera{Int}(2,5.0),car_colors=carcolors)

In [ ]:
td1 = load_trajdata(1);
scene = Scene(500)
get!(scene,td1,300)
display(scene.entities[1].state.posF.s)
roadway = ROADWAY_101;
particle = Dict(:v_des=>25.0,:σ=>0.5)
hallucinate_a_step(roadway,scene,particle,car_id=scene.entities[1].id)

In [ ]:
v_array = [10.,15.,20.,25.,30.]
num_p = length(v_array)
sig_array = [0.1,0.1,0.1,0.1,0.1]	
p_set_dict = Dict(:v_des=>v_array,:σ=>sig_array)
td1 = load_trajdata(1);
scene = Scene(500)
get!(scene,td1,300)
roadway = ROADWAY_101;
trupos = hallucinate_a_step(roadway,scene,Dict(:v_des=>25.0,:σ=>0.0),car_id=4)
lkhd_vec,p_mat,params = compute_particle_likelihoods(roadway,scene,trupos,p_set_dict,car_id=4)

In [ ]:
	@test length(lkhd_vec) == num_p
	@test length(params) == 2
	@test size(p_mat)[1] == 2
	@test size(p_mat)[2] == 5
	@test any(isnan,lkhd_vec) == false